In [1]:
# 기본 작업 경로 설정
import os
notebook_path = os.path.abspath("project_3_git/readme.md")
notebook_dir = os.path.dirname(notebook_path)
os.chdir(notebook_dir)

# 현재 작업 디렉토리 출력
print("Current working directory: ", os.getcwd())

Current working directory:  /mnt/e/py_data/project_3_git


In [2]:
# 텍스트 데이터 가져오기
import json

# JSON 파일에서 딕셔너리 읽기
with open('data/text_data/output_text.json', 'r') as file:
    data_loaded = json.load(file)

data_loaded['0']

['공포, 헤드폰', '아찔한 분위기네요! 🎧 어떤 음악 듣고 계신가요? 궁금해요! 😊']

In [3]:
train_x = [data_loaded[i][0] for i in data_loaded][:5000]
train_y = [data_loaded[i][1] for i in data_loaded][:5000]
test_x = [data_loaded[i][0] for i in data_loaded][5000:]
test_y = [data_loaded[i][1] for i in data_loaded][5000:]

In [6]:
from transformers import T5Config ,T5TokenizerFast, T5ForConditionalGeneration, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
from tqdm import tqdm
import subprocess



name_folder = 'transfer_1'
server_port = '4561'
batch_size = 16
train_epochs = 60



# TensorBoard 서버 실행 (백그라운드)
subprocess.Popen(['tensorboard', f'--logdir=t5/{name_folder}', f'--port={server_port}'])

<Popen: returncode: None args: ['tensorboard', '--logdir=t5/transfer_1', '--...>

2024-09-03 10:37:22.817042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-03 10:37:22.834881: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-03 10:37:22.839826: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-03 10:37:22.852272: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 10:37:23.630554: W tensorflow/compiler/tf2

### 모델 인자 조정
- 드롭 아웃 비율 조정 기본값 : 0.1 -> 0.2
- 학습 률 조정
- 웜 업 스텝 설정
- 배치 사이즈 : 코랩에서 돌릴때 시도
- l2 정규화 (weight decay)
- gredient clipping 그라디언트 조정으로 학습 안정화 시키기
- 레이블 스무딩

In [10]:
# 모델과 토크나이저 로드
tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-base')
config = T5Config.from_pretrained('paust/pko-t5-base')
model = T5ForConditionalGeneration.from_pretrained('paust/pko-t5-base')

config.dropout_rate = 0.2  # 드롭아웃 비율을 20%로 설정 (기본값은 0.1)


# 데이터 전처리 함수 정의
def preprocess_function(examples):
    model_inputs = tokenizer(examples['input_text'], max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(examples['target_text'], max_length=128, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# 데이터셋 생성 및 토큰화
dataset_train = Dataset.from_dict({'input_text': train_x,'target_text': train_y})
dataset_test = Dataset.from_dict({'input_text': test_x,'target_text': test_y})

tokenized_train_datasets = dataset_train.map(preprocess_function, batched=True)
tokenized_test_datasets = dataset_test.map(preprocess_function, batched=True)

# 학습인자
training_args = TrainingArguments(
    per_device_train_batch_size=batch_size, # 학습 배치 사이즈
    per_device_eval_batch_size=batch_size,  # 평가 배치 사이즈
    output_dir=f't5/{name_folder}',         # 모델 및 체크포인트 저장 디렉토리
    num_train_epochs=train_epochs,          # 학습 에폭 수
    logging_dir=f't5/{name_folder}/logs',   # TensorBoard 로그가 저장될 디렉토리
    logging_steps=100,                      # TensorBoard 로그를 기록할 간격 
    report_to='tensorboard',                # TensorBoard로 로깅
    load_best_model_at_end = True,
    eval_strategy='epoch',
    save_strategy='epoch',                  # 에포크 마다 모델 저장
    # resume_from_checkpoint=True           # 이어 학습

    # 추가 인자
    learning_rate=5e-5,                     # 기본값에서 시작
    lr_scheduler_type="linear",             # 스케줄러
    warmup_steps=500,                       # 500 스텝 동안 학습률을 점진적으로 증가
    weight_decay=0.01,                      # l2 정규화 기법 중 하나
    max_grad_norm=1.0,                      # 그라디언트 클리핑
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_test_datasets,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

# 모델 학습
trainer.train() # 이어 학습시 (resume_from_checkpoint=checkpoint_dir)

# 모델과 토크나이저 저장
model_save_path = f't5/{name_folder}/model'
tokenizer_save_path = f't5/{name_folder}/model/tokenizer'

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"모델이 '{model_save_path}'에 저장되었습니다.")
print(f"토크나이저가 '{tokenizer_save_path}'에 저장되었습니다.")

Map: 100%|████████████████████████████████████████████████| 994/994 [00:00<00:00, 12174.41 examples/s]


Epoch,Training Loss,Validation Loss
1,1.400500,0.327589
2,0.254200,0.210854
3,0.217600,0.193800
4,0.197800,0.187665
5,0.187800,0.182515
6,0.181000,0.180372
7,0.173600,0.179033
8,0.168700,0.178390


### 결과 확인

In [20]:
model = T5ForConditionalGeneration.from_pretrained('t5/transfer_0/model/')
tokenizer = T5TokenizerFast.from_pretrained('t5/transfer_0/model/tokenizer/')

In [22]:
import torch

# 테스트 입력
test_input = "야!!!!"

# 입력 토큰화
input_ids = tokenizer.encode(test_input, return_tensors='pt')

# GPU가 사용 가능한지 확인하고, 사용 가능한 경우 GPU로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델과 토크나이저를 GPU로 이동
model.to(device)

# 입력 데이터도 GPU로 이동
input_ids = input_ids.to(device)

# 예측을 GPU에서 수행
with torch.no_grad():
    outputs = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)

# 예측 결과 디코딩
predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"테스트 입력: {test_input}")
print(f"모델의 예측: {predicted_text}")

테스트 입력: 야!!!!
모델의 예측: 정말 화가 나신 것 같아요! 😠 어떤 일이 있으셨나요?
